<a href="https://colab.research.google.com/github/priyanshgupta1998/Networkx/blob/master/twitter_analysis3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#streaming_tweets

#Tweepy   
An easy-to-use Python library for accessing the Twitter API.  
http://www.tweepy.org   
https://tweepy.readthedocs.io/en/v3.5.0/

In [0]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [0]:
ACCESS_TOKEN = " "
ACCESS_TOKEN_SECRET = " "
CONSUMER_KEY = " "
CONSUMER_SECRET = " "

In [0]:
class StdOutListener(StreamListener):
    def on_data(self, data):
        print(data)
        return True
    def on_error(self, status):
        print(status)
if __name__ == '__main__':
    # Authenticate using config.py and connect to Twitter Streaming API.
    listener = StdOutListener()        # Create constructor / object of class StdOutListener
    auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
    # authentication
    stream = Stream(auth, listener)
    hash_tag_list = ["timesofindia"]
    #streaming
    stream.filter(track=hash_tag_list)    # Real time tweets streaming   # just before from 5:30 hour



---



---



---



---



In [0]:
# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        pass

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
        stream = Stream(auth, listener)
        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)

In [0]:
# # # # TWITTER STREAM LISTENER # # # #
class StdOutListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          

    def on_error(self, status):
        print(status)


In [0]:
if __name__ == '__main__':
    # Authenticate using config.py and connect to Twitter Streaming API.
    hash_tag_list = ["donal trump", "hillary clinton", "barack obama", "bernie sanders","timesofindia"]
    fetched_tweets_filename = "/home/tweets.txt"
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)